# Webscraping of top 100 adventure movies on IMDb

Needed information:
* Movie name:
* Description
* Release Date
* Director Name
* Rating
* Duration
* Genre
* Stars (Actors)
* Filming Dates


In [153]:
import requests
from bs4  import BeautifulSoup
import pandas as pd

### Webpage
Each page can be accessed through the `start=` filter in the url. E.g. `start=1` will list movies from 1 to 50 and `start=51` will show the next page (i.e., 51 to 100 top movies)


In [154]:
page_filter = [1, 51]

soup = {}
for start_num in page_filter:
    
    url = f"https://www.imdb.com/search/title/?title_type=feature&genres=adventure&start={start_num}&explore=genres&ref_=adv_nxt"
    response = requests.get(url)
    print(response)
    page = BeautifulSoup(response.content, 'html.parser')
    soup[start_num] = page


<Response [200]>
<Response [200]>


In [155]:
# raw data
raw_data = {

    'movie_name': [],
    'description': [],
    'release_date': [],
    'director': [],
    'rating': [],
    'duration': [],
    'genre': [],
    'actors': [],
    'filming_dates': [],
    'votes': []
    
    }

In [156]:
    
for key in soup:

    # # movie name
    # result = soup[key].select(".lister-item-header a")
    # for tag in result:
    #     raw_data["movie_name"].append(tag.text)


    # # # description
    # result = soup[key].select(".text-muted+ .text-muted , .ratings-bar+ .text-muted")
    # for tag in result:
    #     raw_data["description"].append(tag.text.replace("\n", ''))


    # # # release_date
    # result = soup[key].select(".text-muted.unbold")
    # for tag in result:
    #     raw_data["release_date"].append(tag.text.replace('(', '').replace(')', ''))


    # # director(s)
    # result = soup[key].select(".text-muted~ .text-muted+ p , .ratings-bar~ .text-muted+ p")
    # for tag in result:
    #     raw_data["director"].append(tag.text.strip().replace('\n', '').split('|')[0].split(':')[1])

    # # actors
    # result = soup[key].select(".text-muted~ .text-muted+ p , .ratings-bar~ .text-muted+ p")
    # for tag in result:
    #     raw_data["actors"].append(tag.text.strip().replace('\n', '').split('|')[1].split(':')[1])
 

    # duration (missing some data). So select a parent tag where duration is usually located and then apply
    # str.extract method of pd.Series in combination with regex to extract duration if found and nan if not
    # result = soup[key].select(".lister-item-header+ .text-muted")
    # for tag in result:
    #     duration = pd.Series(tag.text).str.extract(r'(\d+).min').squeeze()
    #     raw_data['duration'].append(duration)


    # rating. Just as duration also some missing data. So, we'll again target a much broader div and extract the rating
    # whenever it's given, otherwise `nan` is returned .lister-item-content
    # result = soup[key].select(".lister-item-content")
    # for tag in result:
    #     rating = pd.Series(tag.text).str.replace('\n','').str.extract(r'(\d\.\d)Rate this').squeeze()
    #     raw_data['rating'].append(rating)

    # votes
    result = soup[key].select(".lister-item-content")
    for tag in result:
        votes = (   pd.Series(tag.text)                 # transform to a pd.Series object to be able to use str.extract
                    .str.replace('\n','')               # remove all new lines
                    .str.extract(r'Votes:(\d+,?\d+)')   # find a pattern where there is 'Votes:' followed by one or more digit,
                                                        # followed by an optional comma (votes below 1000 have no comma sep),  
                                                        # followed by one or more digit, then extract the part in parantheses.
                    .squeeze()                          # the previous step yields a dataframe, transform back to normal string object. 
                                                        # (Normally used to transform one column df back into a series)
                )
        raw_data['votes'].append(votes)

In [158]:
# in total 3 values that had no comma
print(pd.Series(raw_data['votes']).isna().sum())
# raw_data['votes']

17
